In [20]:
from dotenv import load_dotenv
import os
import requests
load_dotenv()
def get_aqi_str(aqi_key:str)->str:
    url = f'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key={aqi_key}&limit=1000&sort=datacreationdate desc&format=JSON'
    try:
        r = requests.get(url)
        r.raise_for_status()
    except Exception as e:
        print(f"發生錯誤:{e}")
    return r.text

aqi_data:str = get_aqi_str(aqi_key=os.environ['AQI_KEY'])

In [28]:
from pydantic import BaseModel,Field,field_validator,field_serializer
from datetime import datetime

class Record(BaseModel):
    站點: str = Field(alias='sitename')
    縣市: str = Field(alias='county')
    日期: datetime = Field(alias='datacreationdate')
    aqi: float | None 
    狀態: str = Field(alias='status')
    pm25: float | None = Field(alias='pm2.5')
    緯度: float = Field(alias='latitude')
    經度: float = Field(alias='longitude')

    @field_validator("pm25","aqi",mode='before')
    @classmethod
    def pm25_for_empty_string(cls,v:str)->float:
        if v == '':
            return None
        else:
            return float(v)
    
    @field_serializer('緯度','經度')
    def four_digits(self,value:float)->float:
        return round(value,ndigits=4)
    
    @field_serializer('日期')
    def datetime_to_string(self,date:datetime) -> str:
        return date.strftime('%Y:%m:%d %H:%M:00')


class AQI(BaseModel):
    records: list[Record]

aqi = AQI.model_validate_json(aqi_data)

ValidationError: 1 validation error for AQI
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='由本平臺呼叫環境...升資料索取權限', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid

In [29]:
aqi_data:list[dict] = aqi.model_dump()['records']
for aqi_item in aqi_data:
    print(aqi_item)

{'站點': '基隆', '縣市': '基隆市', 'aqi': 28, '狀態': '良好', 'pm25': 6.0, '緯度': 25.1292, '經度': 121.7601}
{'站點': '汐止', '縣市': '新北市', 'aqi': 28, '狀態': '良好', 'pm25': 6.0, '緯度': 25.0662, '經度': 121.6408}
{'站點': '萬里', '縣市': '新北市', 'aqi': 31, '狀態': '良好', 'pm25': 10.0, '緯度': 25.1797, '經度': 121.6899}
{'站點': '新店', '縣市': '新北市', 'aqi': 33, '狀態': '良好', 'pm25': 8.0, '緯度': 24.9772, '經度': 121.5378}
{'站點': '土城', '縣市': '新北市', 'aqi': 30, '狀態': '良好', 'pm25': 9.0, '緯度': 24.9825, '經度': 121.4519}
{'站點': '板橋', '縣市': '新北市', 'aqi': 34, '狀態': '良好', 'pm25': 7.0, '緯度': 25.013, '經度': 121.4587}
{'站點': '新莊', '縣市': '新北市', 'aqi': 32, '狀態': '良好', 'pm25': 10.0, '緯度': 25.038, '經度': 121.4325}
{'站點': '菜寮', '縣市': '新北市', 'aqi': 33, '狀態': '良好', 'pm25': 8.0, '緯度': 25.069, '經度': 121.481}
{'站點': '林口', '縣市': '新北市', 'aqi': 31, '狀態': '良好', 'pm25': 3.0, '緯度': 25.078, '經度': 121.3655}
{'站點': '淡水', '縣市': '新北市', 'aqi': 35, '狀態': '良好', 'pm25': 11.0, '緯度': 25.1645, '經度': 121.4492}
{'站點': '士林', '縣市': '臺北市', 'aqi': 33, '狀態': '良好', 'pm25': 1.0, '緯度': 25.1